In [9]:
import pandas as pd
from fuzzywuzzy import fuzz, process

# Load the datasets
df_csv = pd.read_csv("/workspaces/ETL-pipeline/dags/electricity-generation_emissions_sources_ownership.csv")
df_xlsx = pd.read_csv("/workspaces/ETL-pipeline/dags/Global-Nuclear-Power-Tracker-October-2023.csv")

# Print column names to verify
print("DF CSV Columns:\n", df_csv.columns)
print("DF XLSX Columns:\n", df_xlsx.columns)

# Standardize and clean column names
df_csv.columns = df_csv.columns.str.strip().str.lower().str.replace(' ', '_')
df_xlsx.columns = df_xlsx.columns.str.strip().str.lower().str.replace(' ', '_')

# Verify standardized column names
print("Standardized DF CSV Columns:\n", df_csv.columns)
print("Standardized DF XLSX Columns:\n", df_xlsx.columns)

# Extract unique company names from both datasets
company_names_csv = df_csv['company_name'].unique()
company_names_xlsx = df_xlsx['owner'].unique()

# Perform fuzzy matching to find the closest matches between company names
matches = {name: process.extractOne(name, company_names_xlsx, scorer=fuzz.token_set_ratio)[0] for name in company_names_csv}
df_csv['matched_company_name'] = df_csv['company_name'].map(matches)

# Verify some of the matches
print("Some matched company names:\n", df_csv[['company_name', 'matched_company_name']].head())

# Merge the datasets based on the matched company names
df_merged = pd.merge(df_csv, df_xlsx, left_on='matched_company_name', right_on='owner', suffixes=('_csv', '_xlsx'))

# Print the merged DataFrame to verify
print("Merged DataFrame Head:\n", df_merged.head())

# Save the merged DataFrame to a CSV file for further analysis if needed
#df_merged.to_csv("/workspaces/ETL-pipeline/dags/merged_company_data.csv", index=False)


/home/codespace/.python/current/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


DF CSV Columns:
 Index(['source_id', 'source_name', 'iso3_country', 'original_inventory_sector',
       'lat', 'lon', 'geometry_ref', 'relationship', 'ultimate_parent_name',
       'ultimate_parent_id', 'percent_interest_parent', 'company_name',
       'company_id', 'percent_interest_company', 'interest_units',
       'start_date', 'end_date', 'created_date', 'modified_date',
       'percent_company_datasource', 'percent_parent_datasource'],
      dtype='object')
DF XLSX Columns:
 Index(['Date Last Researched', 'Country', 'Project Name', 'Unit Name',
       'Project Name in Local Language / Script', 'Other Name(s)',
       'Capacity (MW)', 'Status', 'Reactor Type', 'Model', 'Start Year',
       'Retirement Year', 'Planned Retirement', 'Cancellation Year',
       'Construction Start Date', 'First Criticality Date',
       'First Grid Connection', 'Commercial Operation Date', 'Retirement Date',
       'Owner', 'Owner Name in Local Language / Script', 'Operator',
       'Operator Name in 

In [10]:
df_merged.to_csv("/workspaces/ETL-pipeline/dags/merged_company_data.csv", index=False)

In [5]:
import pandas as pd
from fuzzywuzzy import fuzz, process

# Load the datasets
df_power = pd.read_csv("/workspaces/ETL-pipeline/dags/electricity-generation_emissions_sources_ownership.csv")
df_nuclear = pd.read_csv("/workspaces/ETL-pipeline/dags/Global-Nuclear-Power-Tracker-October-2023.csv")

# # Clean and standardize column names
# df_nuclear.columns = df_nuclear.columns.str.strip().str.lower().str.replace(' ', '_')
# df_power.columns = df_power.columns.str.strip().str.lower().str.replace(' ', '_')

# # Extract unique company names from both datasets
# company_names_nuclear = df_nuclear['owner'].unique()
# company_names_power = df_power['company_name'].unique()

# # Perform fuzzy matching to find the closest matches between company names
# matches = {name: process.extractOne(name, company_names_nuclear, scorer=fuzz.token_set_ratio)[0] for name in company_names_power}
# df_power['matched_company_name'] = df_power['company_name'].map(matches)

# # Merge the datasets based on the matched company names
# df_merged = pd.merge(df_power, df_nuclear, left_on='matched_company_name', right_on='owner', suffixes=('_power', '_nuclear'))

In [6]:
import pandas as pd
from fuzzywuzzy import fuzz
import datetime

# Function to standardize company names
def standardize_company_name(name):
    return name.lower().replace(' sa', '').replace(' corp', '').strip()

# Function to find the best match for a company name
def find_best_match(name, existing_names, threshold=80):
    best_match = None
    best_score = 0
    for existing_name in existing_names:
        score = fuzz.ratio(standardize_company_name(name), standardize_company_name(existing_name))
        if score > best_score and score >= threshold:
            best_match = existing_name
            best_score = score
    return best_match

# Main processing function
def process_data(nuclear_df, ownership_df):
    # Create empty lists to store processed data
    processed_data = []
    
    # Process nuclear power plant data
    for _, row in nuclear_df.iterrows():
        owners = row['Owner'].split(';')
        for owner in owners:
            parts = owner.split('[')
            parent_company = parts[0].strip()
            percent_interest = float(parts[1].strip('%]')) if len(parts) > 1 else 100.0
            
            processed_data.append({
                'asset_name': row['Project Name'],
                'asset_type': 'Nuclear Power Plant',
                'unit_name': row['Unit Name'],
                'capacity_mw': row['Capacity (MW)'],
                'status': row['Status'],
                'reactor_type': row['Reactor Type'],
                'start_year': row['Start Year'],
                'country': row['Country'],
                'latitude': row['Latitude'],
                'longitude': row['Longitude'],
                'owner': parent_company,
                'percent_interest': percent_interest
            })

    # Process ownership data
    for _, row in ownership_df.iterrows():
        processed_data.append({
            'asset_name': row['source_name'],
            'asset_type': 'Power Plant',
            'unit_name': '',
            'capacity_mw': None,
            'status': None,
            'reactor_type': None,
            'start_year': None,
            'country': row['iso3_country'],
            'latitude': row['lat'],
            'longitude': row['lon'],
            'owner': row['ultimate_parent_name'],
            'percent_interest': row['percent_interest_company']
        })

    # Convert processed data to DataFrame
    result_df = pd.DataFrame(processed_data)
    
    # Add processing timestamp
    result_df['processed_date'] = datetime.datetime.now()

    return result_df

# Run the processing and save to CSV
result_df = process_data(df_nuclear, df_power)
# result_df.to_csv('processed_power_plants.csv', index=False)
print("Data processing complete. Results saved to 'processed_power_plants.csv'")

AttributeError: 'float' object has no attribute 'split'